In [1]:
import time
import pandas as pd
from datetime import datetime
import threading

import bernielib as bl

bl.listSerialPorts()

['COM7', 'COM18', 'COM20']

In [2]:
ber = bl.robot(cartesian_port_name='COM18', pipette_port_name='COM7', misc_port_name='COM20')

In [3]:
ber.robotHome()
ber.pipetteHome()

# Pre-load with ladder

In [4]:
initial_vol_list = [30, 30, 30, 30, 30]
samples_number = len(initial_vol_list)
dna_stock_2 = bl.createSample('eppendorf', 'dna_stock_2', ber.samples_rack, 0, 8, 160)

In [5]:
ber.pickUpNextTip()
samples_list = bl.createSamplesToPurifyList(ber, [0 for x in range(samples_number)])

for sample, volume in zip(samples_list, initial_vol_list):
    ber.transferLiquid(dna_stock_2, sample, volume, v_immerse_dispense=1600, touch_wall=False)

ber.move(z=50)
ber.dumpTipToWaste()
ber.pipetteHome()

# Adding beads

In [6]:
def addBeadsToAll(robot, samples_list, v_beads_list, beads, sample_mix_scenario, beads_mix_scenario):
    robot.moveMagnetsAway(poweroff=True)
    robot.pickUpNextTip()
    robot.move(z=50)
    robot.mixByScript(beads, beads_mix_scenario)
    robot.move(z=50)
    
    for sample, v_beads in zip(samples_list, v_beads_list):
        robot.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
    counter = 0
    for sample, v_beads in zip(samples_list, v_beads_list):
        if counter != 0:
            robot.move(z=50)
            robot.pickUpNextTip()
        
        robot.move(z=50)
        
        robot.mixByScript(sample, sample_mix_scenario)
            
        if counter == 0:
            timestamp = time.time()
        counter += 1
        
        robot.move(z=50)
        robot.dumpTipToWaste()
    
    return timestamp

In [6]:
30*0.9

27.0

In [10]:
V_avail_beads = 800 #uL
v_beads_list = [90, 27, 18, 15]
samples_list = bl.createSamplesToPurifyList(ber, [30, 30, 30, 30])
beads = bl.createSample('eppendorf', 'beads', ber.samples_rack, 0, 11, V_avail_beads)
sample_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')
beads_mix_scenario = pd.read_csv('mixing_pattern_eppendorf.csv')

ber.pickUpNextTip()

ber.move(z=50)
ber.mixByScript(beads, beads_mix_scenario)
ber.move(z=50)

for sample, v_beads in zip(samples_list, v_beads_list):
    ber.transferLiquid(beads, sample, v_beads, touch_wall=False)
    
ber.dumpTipToWaste()

In [8]:
ber.move(z=50)

In [9]:
ber.dumpTipToPosition(8, 0)

# Settings

1. Ladder control
2. Sup from adding 90 uL of beads
3. Sup from adding 27 uL of beads
4. Sup from adding 18 uL of beads
5. Sup from adding 15 uL of beads
6. Eluent-90
7. Eluent-27
8. Eluent-18
9. Eluent-15